Prerequsites: 
pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import csv

Gathering Data from Kaggle.com and store in a dataframe 

In [23]:
od.download("https://www.kaggle.com/datasets/erinkhoo/property-price-register-ireland/data")

propertyPrice_df = pd.read_csv('property-price-register-ireland/Property_Price_Register_Ireland-28-05-2021.csv')
propertyPrice_df

Skipping, found downloaded files in "./property-price-register-ireland" (use force=True to force download)


,SALE_DATE,ADDRESS,POSTAL_CODE,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,PROPERTY_SIZE_DESC
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,343000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
1,2010-01-03,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,0,1,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,438500.0,0,0,Second-Hand Dwelling house /Apartment,NaN
3,2010-01-04,"1 The Haven, Mornington",NaN,Meath,400000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
4,2010-01-04,"11 Melville Heights, Kilkenny",NaN,Kilkenny,160000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
...,...,...,...,...,...,...,...,...,...
476740,2021-05-28,"GRANGE STORES, GRANGE, MULLINGAR",NaN,Westmeath,150000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
476741,2021-05-28,"MAIN STREET, LOWER BORRIS, BORRIS",NaN,Carlow,170000.0,1,0,Second-Hand Dwelling house /Apartment,NaN
476742,2021-05-28,"MULTYFARNHAM, WESTMEATH",NaN,Westmeath,175000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
476743,2021-05-28,"NO 2 RIVERLODGE, SHRONE, GLENGARIFF",NaN,Cork,240000.0,0,0,Second-Hand Dwelling house /Apartment,NaN


Cleaning Dataset

In [24]:
propertyPrice_df.drop(columns=["ADDRESS", "POSTAL_CODE", "IF_MARKET_PRICE", "PROPERTY_SIZE_DESC" ], inplace=True) #storing the new dataframe as 
propertyPrice_df.drop_duplicates() #remove any duplicates from the data set as they are not necesssay for data processing and just take up processing resources




,SALE_DATE,COUNTY,SALE_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC
0,2010-01-01,Dublin,343000.0,0,Second-Hand Dwelling house /Apartment
1,2010-01-03,Laois,185000.0,1,New Dwelling house /Apartment
2,2010-01-04,Dublin,438500.0,0,Second-Hand Dwelling house /Apartment
3,2010-01-04,Meath,400000.0,0,Second-Hand Dwelling house /Apartment
4,2010-01-04,Kilkenny,160000.0,0,Second-Hand Dwelling house /Apartment
...,...,...,...,...,...
476740,2021-05-28,Westmeath,150000.0,0,Second-Hand Dwelling house /Apartment
476741,2021-05-28,Carlow,170000.0,0,Second-Hand Dwelling house /Apartment
476742,2021-05-28,Westmeath,175000.0,0,Second-Hand Dwelling house /Apartment
476743,2021-05-28,Cork,240000.0,0,Second-Hand Dwelling house /Apartment


In [6]:
# Count the number of rows for each value in IF_MARKET_PRICE
#market_price_counts = df['IF_MARKET_PRICE'].value_counts()

# Count the number of rows for each value in IF_VAT_EXCLUDED
vat_excluded_counts = df['IF_VAT_EXCLUDED'].value_counts()

#print("Count of each value in IF_MARKET_PRICE:")
#print(market_price_counts)

print("\nCount of each value in IF_VAT_EXCLUDED:")
print(vat_excluded_counts)


Count of each value in IF_VAT_EXCLUDED:
IF_VAT_EXCLUDED
0    400648
1     76097
Name: count, dtype: int64


if vat exclusive add 13.5% 

In [7]:
# Update the 'House_Price' column based on the 'IF_VAT_EXCLUDED' column
df.loc[df['IF_VAT_EXCLUDED'] == 1, 'SALE_PRICE'] = df.loc[df['IF_VAT_EXCLUDED'] == 1, 'SALE_PRICE'] + (df['SALE_PRICE'] * 0.135)

# Display the updated DataFrame
print(df)

         SALE_DATE     COUNTY  SALE_PRICE  IF_VAT_EXCLUDED  \
0       2010-01-01     Dublin    343000.0                0   
1       2010-01-03      Laois    209975.0                1   
2       2010-01-04     Dublin    438500.0                0   
3       2010-01-04      Meath    400000.0                0   
4       2010-01-04   Kilkenny    160000.0                0   
...            ...        ...         ...              ...   
476740  2021-05-28  Westmeath    150000.0                0   
476741  2021-05-28     Carlow    170000.0                0   
476742  2021-05-28  Westmeath    175000.0                0   
476743  2021-05-28       Cork    240000.0                0   
476744  2021-05-28      Cavan     47000.0                0   

                                PROPERTY_DESC  
0       Second-Hand Dwelling house /Apartment  
1               New Dwelling house /Apartment  
2       Second-Hand Dwelling house /Apartment  
3       Second-Hand Dwelling house /Apartment  
4       Second-

In [8]:
# Rename the 'OldColumnName' to 'NewColumnName'
df = df.rename(columns={'PROPERTY_DESC': 'New Build'})

In [9]:
df.loc[df['New Build'] == 'New Dwelling house /Apartment', 'New Build'] = '1'
df.loc[df['New Build'] == 'Second-Hand Dwelling house /Apartment', 'New Build'] = '0'

df 

,SALE_DATE,COUNTY,SALE_PRICE,IF_VAT_EXCLUDED,New Build
0,2010-01-01,Dublin,343000.0,0,0
1,2010-01-03,Laois,209975.0,1,1
2,2010-01-04,Dublin,438500.0,0,0
3,2010-01-04,Meath,400000.0,0,0
4,2010-01-04,Kilkenny,160000.0,0,0
...,...,...,...,...,...
476740,2021-05-28,Westmeath,150000.0,0,0
476741,2021-05-28,Carlow,170000.0,0,0
476742,2021-05-28,Westmeath,175000.0,0,0
476743,2021-05-28,Cork,240000.0,0,0


In [10]:
df.drop(columns=["IF_VAT_EXCLUDED"], inplace=True) #storing the new dataframe 
df

,SALE_DATE,COUNTY,SALE_PRICE,New Build
0,2010-01-01,Dublin,343000.0,0
1,2010-01-03,Laois,209975.0,1
2,2010-01-04,Dublin,438500.0,0
3,2010-01-04,Meath,400000.0,0
4,2010-01-04,Kilkenny,160000.0,0
...,...,...,...,...
476740,2021-05-28,Westmeath,150000.0,0
476741,2021-05-28,Carlow,170000.0,0
476742,2021-05-28,Westmeath,175000.0,0
476743,2021-05-28,Cork,240000.0,0


Calculating Mean, Mode and median for the whole data set

In [11]:
from statistics import mode, median

In [12]:
mean = df["SALE_PRICE"].mean()
mode_value = mode(df["SALE_PRICE"])
median_value = median(df["SALE_PRICE"])

print("Mean SALE_PRICE:", mean)
print("Mode SALE_PRICE:", mode_value)
print("Median SALE_PRICE:", median_value)

Mean SALE_PRICE: 265525.12517328333
Mode SALE_PRICE: 150000.0
Median SALE_PRICE: 200000.0


In [13]:
# Assuming you have a DataFrame 'df' with columns 'COUNTY' and 'SALE_PRICE'

# Calculate the mean per county
mean_per_county = df.groupby('COUNTY')['SALE_PRICE'].mean()

# Calculate the mode per county
mode_per_county = df.groupby('COUNTY')['SALE_PRICE'].apply(lambda x: mode(x) if len(x) > 0 else None)

# Calculate the median per county
median_per_county = df.groupby('COUNTY')['SALE_PRICE'].median()

# Create a new DataFrame with the results
summary_df = pd.DataFrame({'Mean_SALE_PRICE': mean_per_county, 'Mode_SALE_PRICE': mode_per_county, 'Median_SALE_PRICE': median_per_county})

# Print or use summary_df as needed
print(summary_df)

           Mean_SALE_PRICE  Mode_SALE_PRICE  Median_SALE_PRICE
COUNTY                                                        
Carlow       156838.534364         120000.0          140000.00
Cavan        121279.239565          50000.0          100000.00
Clare        161892.103215         150000.0          136000.00
Cork         233781.501401         200000.0          196000.00
Donegal      119849.301731         100000.0          100000.00
Dublin       421953.904551         250000.0          310000.00
Galway       208655.519992         150000.0          176000.00
Kerry        163813.994920         100000.0          140000.00
Kildare      278252.045974         200000.0          255000.00
Kilkenny     185259.009902         150000.0          160000.00
Laois        154778.246811         100000.0          135000.00
Leitrim      103595.642527          50000.0           85000.15
Limerick     173230.855396         150000.0          150000.00
Longford     100088.249710         100000.0           8

In [14]:
# Extract the year from the 'SALE_DATE' column
df['SALE_YEAR'] = pd.to_datetime(df['SALE_DATE']).dt.year

# Group by 'COUNTY' and 'SALE_YEAR'
grouped = df.groupby(['COUNTY', 'SALE_YEAR'])

# Calculate the mean, mode, and median per county and year
mean_per_county_year = grouped['SALE_PRICE'].mean()
mode_per_county_year = grouped['SALE_PRICE'].apply(lambda x: mode(x) if len(x) > 0 else None)
median_per_county_year = grouped['SALE_PRICE'].median()

# Create a new DataFrame with the results
summary_df = pd.DataFrame({'Mean_SALE_PRICE': mean_per_county_year, 'Mode_SALE_PRICE': mode_per_county_year, 'Median_SALE_PRICE': median_per_county_year}).reset_index()

# Print or use summary_df as needed
print(summary_df)

      COUNTY  SALE_YEAR  Mean_SALE_PRICE  Mode_SALE_PRICE  Median_SALE_PRICE
0     Carlow       2010    183103.892963        158999.88        169999.1650
1     Carlow       2011    143834.613309        170000.00        129950.0000
2     Carlow       2012    111141.007784        100000.00        100000.0000
3     Carlow       2013    115669.161800         35000.00         90000.9600
4     Carlow       2014    117306.016439        150000.00        112000.0000
..       ...        ...              ...              ...                ...
307  Wicklow       2017    340932.792446        300000.00        299999.9883
308  Wicklow       2018    343802.121295        310000.00        302000.0000
309  Wicklow       2019    370105.657973        250000.00        310000.0000
310  Wicklow       2020    378801.979148        300000.00        327000.0000
311  Wicklow       2021    416200.267424        330000.00        364646.1200

[312 rows x 5 columns]


In [21]:
display(summary_df.to_string())
df.to_csv(r'property-price-register-ireland/export_dataframe.csv', index=False, header=True)

print(df)

'        COUNTY  SALE_YEAR  Mean_SALE_PRICE  Mode_SALE_PRICE  Median_SALE_PRICE\n0       Carlow       2010    183103.892963     158999.88000      169999.165000\n1       Carlow       2011    143834.613309     170000.00000      129950.000000\n2       Carlow       2012    111141.007784     100000.00000      100000.000000\n3       Carlow       2013    115669.161800      35000.00000       90000.960000\n4       Carlow       2014    117306.016439     150000.00000      112000.000000\n5       Carlow       2015    130453.348428     120000.00000      120000.000000\n6       Carlow       2016    140306.120846      80000.00000      135000.000000\n7       Carlow       2017    150385.207451     120000.00000      140000.000000\n8       Carlow       2018    174465.904231     100000.00000      165000.000000\n9       Carlow       2019    179657.019430     150000.00000      170000.000000\n10      Carlow       2020    213992.577366     150000.00000      175000.000000\n11      Carlow       2021    201765.552

         SALE_DATE     COUNTY  SALE_PRICE New Build  SALE_YEAR
0       2010-01-01     Dublin    343000.0         0       2010
1       2010-01-03      Laois    209975.0         1       2010
2       2010-01-04     Dublin    438500.0         0       2010
3       2010-01-04      Meath    400000.0         0       2010
4       2010-01-04   Kilkenny    160000.0         0       2010
...            ...        ...         ...       ...        ...
476740  2021-05-28  Westmeath    150000.0         0       2021
476741  2021-05-28     Carlow    170000.0         0       2021
476742  2021-05-28  Westmeath    175000.0         0       2021
476743  2021-05-28       Cork    240000.0         0       2021
476744  2021-05-28      Cavan     47000.0         0       2021

[476745 rows x 5 columns]
